In [ ]:
import os
import numpy as np
from gluoncv import model_zoo, data, utils
from gluoncv.utils.metrics.voc_detection import VOC07MApMetric
from mxnet import gluon
import mxnet as mx

# Load the pretrained model
model_name = 'yolo3_darknet53_voc'
net = model_zoo.get_model(model_name, pretrained=True)

# Load the VOC2012 dataset
val_dataset = data.VOCDetection(root='./VOCdevkit', splits=[(2012, 'val')])

# Define the validation data loader
batch_size = 1
val_loader = gluon.data.DataLoader(
    val_dataset.transform(data.transforms.presets.yolo.YOLO3DefaultValTransform(width=416, height=416)),
    batch_size=batch_size, shuffle=False, last_batch='keep', num_workers=10)

# Define the mAP metric
metric = VOC07MApMetric(iou_thresh=0.5, class_names=val_dataset.classes)

# Run the model on the dataset and compute mAP
print(len(val_loader))
i = 0
for batch in val_loader:
    data = gluon.utils.split_and_load(batch[0], ctx_list=[mx.cpu(0)], batch_axis=0, even_split=False)
    label = gluon.utils.split_and_load(batch[1], ctx_list=[mx.cpu(0)], batch_axis=0, even_split=False)
    det_bboxes = []
    det_ids = []
    det_scores = []
    gt_bboxes = []
    gt_ids = []
    gt_difficults = []
    for x, y in zip(data, label):
        # get prediction results
        ids, scores, bboxes = net(x)
        det_ids.append(ids)
        det_scores.append(scores)
        # clip to image size
        det_bboxes.append(bboxes.clip(0, batch[0].shape[2]))
        # split ground truths
        gt_ids.append(y.slice_axis(axis=-1, begin=4, end=5))
        gt_bboxes.append(y.slice_axis(axis=-1, begin=0, end=4))
        gt_difficults.append(y.slice_axis(axis=-1, begin=5, end=6) if y.shape[-1] > 5 else None)

    # update metric
    metric.update(det_bboxes, det_ids, det_scores, gt_bboxes, gt_ids, gt_difficults)
    if i % 100 == 0:
        print(i)
    i += 1

print('The mAP of the model on the VOC2012 validation dataset is:', metric.get())


/opt/conda/lib/python3.10/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.6.0` and `torch==1.13.0+cu117` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


5823
0
